In [1]:
import math
import numpy as np
import numba as nb

In [2]:
def sample_spherical(npoints, ndim=3):
    vec = np.random.randn(ndim, npoints)
    vec /= np.linalg.norm(vec, axis=0)
    return vec

def sph2cart(obsPolar):  # r=1
    x = np.sin(obsPolar[0]) * np.cos(obsPolar[1])
    y = np.sin(obsPolar[0]) * np.sin(obsPolar[1])
    z = np.cos(obsPolar[0])
    return (x, y, z)

def abberation_angle(angle,obsSpeed):
    abberationAngle = np.arccos((obsSpeed + np.cos(angle))/(obsSpeed*np.cos(angle)+1)) # changed from math.acos to np trig
    return abberationAngle

def unit_norm_vector(v1,v2):
    normVector = np.cross(v1,v2)
    unitNormVector = normVector/np.linalg.norm(normVector)
    return unitNormVector

@nb.njit(fastmath=True, error_model="numpy", parallel=False)
def angle(v1, v2):
    #Check the dimensions, this may also have an effect on SIMD-vectorization
    assert v1.shape[1] == 3
    assert v2.shape[1] == 3
    res = np.empty(v1.shape[0])

    for i in nb.prange(v1.shape[0]):  # v2 is now the observer vector
        dot = 0.
        a = 0.
        b = 0.
        for j in range(3):
            dot += v1[i, j]*v2[0, j]
            a += v1[i, j]**2
            b += v2[0, j]**2
        res[i] = np.arccos(dot/(np.sqrt(a*b)))
        # for j in range(3):
        #     dot+=v1[i,j]*v2[i,j]
        #     a+=v1[i,j]**2
        #     b+=v2[i,j]**2
        # res[i]=np.arccos(dot/(np.sqrt(a*b)))
    return res

def fastRotateMatrixNew(axis, theta):
    # from top post in
    # https://stackoverflow.com/questions/6802577/rotation-of-3d-vector
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    axis = axis / np.linalg.norm(axis,axis=1)[:,np.newaxis] # vectorised yes
    a = np.cos(theta / 2.0)
    vals = -axis * np.sin(theta / 2.0)[:,np.newaxis]
    b = vals[:,0]
    c = vals[:,1]
    d = vals[:,2]
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                     [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                     [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])

def fastRotateMatrixOld(axis,theta):
    # from top post in
    # https://stackoverflow.com/questions/6802577/rotation-of-3d-vector
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    axis = axis / math.sqrt(np.dot(axis, axis))
    a = math.cos(theta / 2.0)
    b, c, d = -axis * math.sin(theta / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                     [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                     [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])

In [29]:
n = 10**7

obsSpeed = 0.001
obsPolar = (np.pi/2 - 30*np.pi/180, np.pi/2)  # pol, az nominally 0.7,4
observerVector = sph2cart(obsPolar)
observerVector2 = np.asarray([observerVector])

xi, yi, zi = sample_spherical(n)
initialPointsVectors = np.asarray([xi, yi, zi]).T
angles = angle(initialPointsVectors,observerVector2)
deltaAngles = -(abberation_angle(angles, obsSpeed) - angles)
rotAxes = unit_norm_vector(np.asarray((xi,yi,zi)).T,observerVector2)

rotatedPoints = np.zeros((len(xi),3))
print('here')

rotationMatrices = fastRotateMatrixNew(rotAxes,deltaAngles)
print(np.shape(rotationMatrices))
# rotatedPoints = np.dot(rotationMatrices,(xi,yi,zi))

print('old slow method now...')
for i in range(0, len(rotatedPoints)):
    rotationMatrices2 = fastRotateMatrixOld(rotAxes[i],deltaAngles[i])
    rotatedPoints[i] = np.dot(rotationMatrices2,(xi[i], yi[i], zi[i]))

# print()

here
(3, 3, 10000000)
old slow method now...


In [30]:
# print(rotationMatrices2)
# print(rotationMatrices[:,:,-1])

# np.dot(rotationMatrices,np.asarray([xi,yi,zi]).T)
# np.matmul(rotationMatrices,np.asarray([xi,yi,zi]).T)
# print(np.einsum('ijk,hj->kj',rotationMatrices,np.asarray([xi,yi,zi]).T))

# rotatedPoints2 = np.zeros((len(xi),3))
# for i in range(0,n):
#     rotatedPoints2[i,:] = np.dot(rotationMatrices[:,:,i],(xi[i], yi[i], zi[i]))

# rotatedPoints2 = np.tensordot(rotationMatrices,np.asarray([xi,yi,zi]).T,axes=1)
rotatedPoints2 = np.einsum('ijk,kj->ki',rotationMatrices,np.asarray([xi,yi,zi]).T)

rotatedPoints2 == rotatedPoints


array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       ...,
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [46]:
axis = rotAxes

# axis2 = axis / math.sqrt(np.dot(axis, axis))

# axis3 = axis / math.sqrt(np.einsum('ij,kj->i',axis,axis))
np.sqrt(np.einsum('ij,ij->i',axis,axis))

array([0.00011436, 0.00036969, 0.0003861 , ..., 0.00038649, 0.00021508,
       0.00017742])

In [25]:
math.sqrt(np.dot(axis[3],axis[3]))

0.07036777293114156

In [88]:
A = [[1,2,3],[4,5,6],[7,8,9]]
B = [1,2,3]
np.dot(A,B)

array([14, 32, 50])

In [15]:
a = np.arange(25).reshape(5,5)
b = np.arange(5)
c = np.arange(6).reshape(2,3)

In [107]:
print(a)
print(b)
print(c)

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]]
[0 1 2 3 4]
[[0 1 2]
 [3 4 5]]


In [16]:
np.dot(a,b)

array([ 30,  80, 130, 180, 230])

In [157]:
rotAxes / np.linalg.norm(rotAxes,axis=1)[:,np.newaxis]

array([[ 0.81059527, -0.2928034 ,  0.50715036],
       [ 0.97415167,  0.11294747, -0.19563075]])

In [156]:
np.linalg.norm(rotAxes,axis=1)[:,np.newaxis]

array([[0.74611465],
       [0.66581749]])

In [195]:
np.asarray([xi,yi,zi]).T

array([[-0.2178238 , -0.16448407, -0.96202795],
       [ 0.15922211, -0.9102752 ,  0.38216145],
       [-0.36594658,  0.55138452, -0.74970541],
       ...,
       [ 0.36296316,  0.93135704, -0.028841  ],
       [-0.43633397, -0.05668245, -0.89799764],
       [-0.05726274, -0.90700167,  0.41721572]])

In [15]:
(np.asarray([xi,yi,zi]).T)

array([[-0.15422132,  0.98522069, -0.07453848],
       [ 0.93082965, -0.22378097, -0.288926  ],
       [-0.49015541, -0.82408287, -0.28396319],
       [-0.10440322, -0.95124145,  0.29024072],
       [-0.00689462,  0.86991227, -0.4931583 ]])

In [34]:
print(rotationMatrices2)
print((xi[1],yi[1],zi[1]))
np.dot(rotationMatrices2,(xi[1], yi[1], zi[1]))

[[ 1.00000000e+00  5.97088615e-06  3.44386300e-06]
 [-5.96791592e-06  9.99999629e-01 -8.61825435e-04]
 [-3.44900759e-06  8.61825415e-04  9.99999629e-01]]
(0.9308296536251841, -0.22378096583762, -0.2889260030886832)


array([ 0.93082732, -0.22353743, -0.28912197])